<a href="https://colab.research.google.com/github/chene/ARQOPUS/blob/main/point_Procrustes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Orthogonal Procrustes Analysis (OPA)

<a href="mailto:chene@robarts.ca?subject=ARQOPUS/OPA">If you have any questions about this page, feel free to send me a note.</a>

[Procrustes](https://en.wikipedia.org/wiki/Procrustes) Analysis refers to the process of transforming one set of data such that it matches better with another. Within the context of computer-assisted interventions (CAI), we are typically interested in **scaling**, followed by **rotation** and **translation** of one set of 3D points so they are *registered* (aligned) with another set of 3D points. *Orthogonal* implies that there is no shearing. We assume there is a 1:1 correspondance between theset two datasets.

Perhaps the authoritative literature on this subject matter is the book by Gower and Dijksterhuis:

```
@book{GD2004,
address   = {New York},
author    = {Gower, John C. and Dijksterhuis, Garmt B.},
publisher = {Oxford University Press},
title     = {Procrustes Problems},
year      = {2004}, }
```

## OPA with isotropic scaling

Orthogonal Procrustes Analysis with isotropic scaling, better known as fiducial (or landmark) based registration in CAI, is described by the following equation:

\begin{equation}
y_i = R a x_i + t + n_i
\end{equation}
where $y_i$ and $x_i$ are $3 \times 1$ vectors representing corresponding points in two spaces, $R_{3 \times 3}$ orthonormal rotation, $t_{3 \times 1}$ translation, $a$ is a scaling factor, and $n_i$ a noise vector. Note that when $a$ is a scalar, it is immaterial on which side of $R$ the scaling is applied.

To solve for $R$, $t$, and $a$, we treat it as a least-squares regression problem and minimize the following cost function:

\begin{equation}
FRE^{2} = \frac{1}{N} \sum_{1}^{N} | R x_i  + t - y_i |^{2}
\end{equation}
where the residual to minimize is called Fiducial Registration Error (FRE). 

The Procrustes Analysis is universal in many field, with some of the earliest solution been proposed as early as 1950's. The two common solutions used in the CAI field, both are closed-form solution and equivalent to each other, are:

```
@article{AHB1987,
author   = {Arun, K. S. and Huang, T. S. and Blostein, Steven D.},
journal  = {IEEE Transactions on Pattern Analysis and Machine Intelligence},
title    = {{Least-Squares Fitting of Two 3-D Point Sets}},
number   = {5},
pages    = {698--700},
volume   = {PAMI-9},
year     = {1987}, 
doi      = {10.1109/TPAMI.1987.4767965}, }
```
with their solution based on Singular Value Decomposition (SVD), and
```
@article{Horn1987,
author   = {Horn, Berthold K. P.},
journal  = {Journal of the Optical Society of America A},
title    = {{Closed-form solution of absolute orientation using unit quaternions}}
number   = {4},
pages    = {629--642},
volume   = {4},
year     = {1987}, 
doi      = {10.1364/JOSAA.4.000629}, }
```
with his solution based on unit quaternion. Note that the SVD solution does **not** solve for the scaling factor, while Horn provides a more generic solution that i) resolves an invertable scaling factor, and ii) a weighted scheme.  Thus, I personally prefer the quaternion formulation.

It should further be noted that the solution based on quaternion formulation works only in $3$D, where the SVD solution is universal to $m$D. 

Here is the Matlab code that implements the closed form solution. Note that we solve for the rotation using the SVD formulation, but the scaling using Horn's formulation:
```
function [R, t, a] = OPA( X, Y )
% Solves Y = R a X + t, least-square solution
% INPUTS:  X : (mxn) mD coordinates,
%          Y : (mxn) mD coordinates
% OUTPUTS: R : mxm orthonormal rotation
%          t : mx1 translation
%          a : a scalar scaling factor
%
% Elvis C.S. Chen
% chene AT robarts DOT ca

[m,n] = size(X); e = ones(n,1); II = eye(n)-((e*e')./n);
[U,~,V] = svd(Y*II*X'); D = eye(m); D(m,m)=det(U*V');
R = U * D * V';                                      % rotation
a = sqrt( sum(sum((Y*II).^2))/sum(sum((X*II).^2)) ); % scaling
t = mean( Y-R*a*X, 2);                               % translation
```

And our equivalent Python implementation is:

In [ ]:
import numpy as np
import math

In [ ]:
def OPA( X, Y ):
# Solves Y = R a X + t, least-square solution
# INPUTS:  X : (mxn) mD coordinates,
#          Y : (mxn) mD coordinates
# OUTPUTS: R : mxm orthonormal rotation
#          t : mx1 translation
#          a : a scalar scaling factor
#
# Elvis C.S. Chen
# chene AT robarts DOT ca
  [m,n] = X.shape
  II = np.identity(n) - np.ones((n,n))/n
  u, s, vh = np.linalg.svd( np.matmul( np.matmul( Y, II ), np.transpose(X) ) )
  D = np.identity(m)
  D[m-1,m-1] = np.linalg.det( np.matmul( u, vh ) )
  R = np.matmul(u, np.matmul(D, vh) ) # rotation
  a = math.sqrt( np.sum ( np.square( np.matmul( Y, II ) ) ) / np.sum( np.square( np.matmul( X, II ) ) ) ) # isotropic scaling
  t = np.reshape(np.mean( Y - np.matmul( R, X * a),1 ), [m,1]) # translation
  return [R, t, a]

We can test the above code using the following codes.

First, let's re-use some of the codes from the earlier example for the generation of rotation matrices:

In [ ]:
def rotx3x3(angle):
    """
    
    Parameters
    ----------
    angle : 
        angle in radian

    Returns
    -------
    a 3x3 rotation matrix about x-axis by an amount "angle"

    """
    ca = math.cos(angle)
    sa = math.sin(angle)
    m = np.array([[1,0,0],[0, ca, -sa], [0, sa, ca]])
    return m
  
def roty3x3(angle):
    """
    
    Parameters
    ----------
    angle : 
        angle in radian

    Returns
    -------
    a 3x3 rotation matrix about y-axis by an amount "angle"

    """
    ca = math.cos(angle)
    sa = math.sin(angle)
    m = np.array([[ca,0,sa],[0,1,0],[-sa,0,ca]])
    return m

def rotz3x3(angle):
    """

    Parameters
    ----------
    angle : 
        angle in radian

    Returns
    -------
    a 3x3 rotation matrix about z-axis by an amoung "angle"

    """
    ca = math.cos(angle)
    sa = math.sin(angle)
    m = np.array([[ca, -sa, 0], [sa, ca, 0], [ 0, 0, 1]])
    return m

### Test case

In the following test, we will first generate the following
- a random scaling factor,
- a random rotational matrix, and
- a random translation
- a random 3x10 3D points ($x_i$).

We generate a ground truth ($y_i$) by applying these transformations, then use our OPA code to recover these transformations.

In [ ]:
dim = 3
n = 10

X = np.random.rand( dim, n )
gt_a = np.random.rand(1,1)
gt_R = np.matmul( np.matmul( roty3x3( np.random.rand(1,1) ), rotx3x3( np.random.rand(1,1) ) ), rotz3x3( np.random.rand(1,1) ) )
gt_t = np.random.rand(3,1) * 10
print('Ground truth rotation:\n', gt_R, '\nGround truth translation\n', gt_t, '\nGround truth scaling\n', gt_a)

e = np.ones((1,n))
Y = np.matmul( gt_a*gt_R, X ) + np.matmul(gt_t,e)

#print(X)
#print(Y)
[R,t,a] = OPA(X,Y)
print('Computed rotation:\n', R, '\nComputed translation\n', t, '\nComputed scaling\n',a)

## OPA with anisotropic scaling

While the OPA with scalar (isotropic) scaling factor has a closed form solution, OPA with anisotropic scaling, i.e. scaling factor is different for each axis, do not.  With anisotropic scaling, we also need to different between **pre-scaling** or **post-scaling**, as the order of the operation (i.e. scaling before rotation, or after) matters.

Dosse and Ten Berge made a break-through in 2010. Using the Majorization Principle, they demonstrated that a monotonically convergent solution is possible:
```
@article{BD2010,
author   = {{Bennani Dosse}, Mohammed and {Ten Berge}, Jos},
journal  = {Journal of Classification},
number   = {1},
pages    = {111--128},
title    = {{Anisotropic Orthogonal Procrustes Analysis}},
volume   = {27},
year     = {2010},
doi      = {10.1007/s00357-010-9046-8}, }
```

Based on their derivation, I presented the following solution to the following problem:
\begin{equation}
FRE^{2} = \frac{1}{N} \sum_{i=1}^{N} | R A x_i + t - y_i |^2
\end{equation}
where $A$ is a diagonal scaling matrix. Note that, in general, $R A != A R$. The solution was first presented at:

```
@inproceedings{CMJ+2014,
author    = {Chen, Elvis C. S. and McLeod, A. Jonathan and Jayarathne, Uditha L. and Peters, Terry M.},
booktitle = {Proc. SPIE 9036, Medical Imaging 2014: Image-Guided Procedures, Robotic Interventions, and Modeling},
editor    = {Yaniv, Ziv R. and Holmes, David R.},
pages     = {90361Z},
title     = {{Solving for free-hand and real-time 3D ultrasound calibration with anisotropic orthogonal Procrustes analysis}},
year      = {2014},
doi       = {10.1117/12.2043080}, }
```

The Matlab code is:
```
function [ R, t, A ] = AOPA_major( X, Y, tol ) % Majorization
% Solves Y = R A X + t, least-square solution
% INPUTS:  X : (mxn) mD coordinates,
%          Y : (mxn) mD coordinates
% OUTPUTS: R : mxm orthonormal rotation
%          t : mx1 translation
%          A : mxm diagonal scaling
%
% Elvis C.S. Chen
% chene AT robarts DOT ca

n = size(X,2); e = ones(n,1); II=eye(n)-((e*e')./n);
mX= normr(X*II); mY = (Y*II);
B = mY*mX'; [U,~,V] = svd( B );
R=U*[1 0 0; 0 1 0; 0 0 det(U*V')]*V';
err = +Inf; E_old = 10000*ones(3,n);
while err > tol
    [U,~,V] = svd( B*diag(diag(R'*B)) );
    R = U*[1 0 0; 0 1 0; 0 0 det(U*V')]*V';  
    E = R*mX-mY; 
    err = norm( E-E_old,'fro' ); E_old = E;
end
B = Y*II*X'; A = diag( diag(B'*R) ./ diag(X*II*X') );
t = mean( Y-R*A*X, 2 );
```

The Python code is:

In [ ]:

def AOPA_Major(X, Y, tol):
    """
    Computes the Procrustean fiducial registration between X and Y with 
    anisotropic Scaling:
        
        Y = R * A * X + t
        
    where X is a mxn matrix, m is typically 3
          Y is a mxn matrix, same dimension as X
          
          R is a mxm rotation matrix,
          A is a mxm diagonal scaling matrix, and
          t is a mx1 translation vector
          
    based on the Majorization Principle

    Elvis C.S. Chen
    chene AT robarts DOT ca
    """
    [m,n] = X.shape
    II = np.identity(n) - np.ones((n,n))/n
    mX = np.nan_to_num(np.matmul(X,II)/np.linalg.norm(np.matmul(X,II), ord=2, axis=1, keepdims=True))
    mY = np.matmul(Y,II)
    
    # estimate the initial rotation
    B = np.matmul(mY, mX.transpose())
    u, s, vh = np.linalg.svd(B)
    
    # check for flip
    D = np.identity(m)
    D[m-1,m-1] = np.linalg.det(np.matmul(u,vh))
    R = np.matmul(u, np.matmul(D,vh))
    
    # loop
    err = np.Infinity
    E_old = 1000000 * np.ones((m,n))
    while err > tol: 
        u, s, vh = np.linalg.svd( np.matmul(B, np.diag(np.diag(np.matmul(R.transpose(),B)))) )
        D[m-1,m-1] = np.linalg.det(np.matmul(u,vh))
        R = np.matmul(u, np.matmul(D,vh))
        E = np.matmul(R,mX) - mY
        err = np.linalg.norm(E-E_old)
        E_old = E
    # after rotation is computed, compute the scale
    B = np.matmul(Y, np.matmul(II, X.transpose()))
    A = np.diag( np.divide( np.diag( np.matmul(B.transpose(), R)), np.diag( np.matmul(X, np.matmul(II, X.transpose()))) ) )
    if (math.isnan(A[2,2])):
        # special case for ultrasound calibration, where z=0
        A[2,2] = .5 * (A[0,0] + A[1,1]) # artificially assign a number to the scale in z-axis
    # calculate translation
    t = np.reshape(np.mean( Y - np.matmul(R, np.matmul(A,X)), 1), [m,1])
    return[R,t,A]

### Test case
In the following test, we will first generate the following
- a random anisotropic scaling factors,
- a random rotational matrix, and
- a random translation
- a random 3x10 3D points ($x_i$).

We generate a ground truth ($y_i$) by applying these transformations, then use our OPA code to recover these transformations.

In [ ]:
dim = 3
n = 10

X = np.random.rand( dim, n )
gt_A = np.identity(dim) * np.random.rand(dim,1)
gt_R = np.matmul( np.matmul( roty3x3( np.random.rand(1,1) ), rotx3x3( np.random.rand(1,1) ) ), rotz3x3( np.random.rand(1,1) ) )
gt_t = np.random.rand(3,1) * 10
print('Ground truth rotation:\n', gt_R, '\nGround truth translation\n', gt_t, '\nGround truth scaling\n', gt_A)

e = np.ones((1,n))
Y = np.matmul( np.matmul(gt_R, gt_A), X ) + np.matmul(gt_t,e)

#print(X)
#print(Y)
[R,t,A] = AOPA_Major(X, Y, 1e-9)
print('Computed rotation:\n', R, '\nComputed translation\n', t, '\nComputed scaling\n',A)